# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트
- 부트스트랩 샘플 사용

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)  # 100개의 결정 트리를 사용하므로 모든 코어를 사용하는 것이 좋음
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #훈련 세트에 대한 점수도 반환

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)  # 특성 중요도

[0.23167441 0.50039841 0.26792718]


특성을 랜덤하게 선택하여 훈련하기 때문에 특정 특성에 과도하게 집중하지 않고 더 많은 특성이 훈련할 기회를 갖는다 >> 과대적합은 줄이고 성능은 높임

#### OOB샘플(out of bag : 부트스트랩 샘플에 포함되지 않고 남는 샘플) 로 부트스트랩 샘플 훈련 결정트리를 자체 평가할 수 있다

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
- 부트스트랩 샘플을 사용하지 않고 결정 트리를 만들 때 전체 훈련 세트를 사용하고 `노드를 무작위로 분할`한다
- 무작위성이 커 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야함. 하지만 (최적의 분할을 찾지 않고) 랜덤하게 노드를 분할하기 때문에 빠르다

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)

print(et.feature_importances_)  # 특성 중요도

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
- 깊이가 `얕은 결정 트리를 연속하여 추가`함 >> 과대적합에 강하다! (기본적으로 깊이 3, 결정트리 100개 사용)
- 경사 하강법을 사용해 트리를 앙상블에 추가함
- 분류에서는 로지스틱 손실함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리 개수를 늘려도 과대적합에 강함!

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)  # 결정 트리 개수 500개
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)  # , 학습률 기본값 0.1

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


그레이디언트 부스팅이 렌덤 포레스트보다 일부 특성에 더 집중한다

## 히스토그램 기반 부스팅
- `입력 특성을 256개의 구간으로` 나눈다 >> 최적의 노드 분할을 빠르게 찾을 수 있다
- 256개 구간 중 하나를 떼어놓고 누락된 값을 위해 사용함. 누락된 특성이 있어도 전처리할 필요 없음
- 트리의 개수를 지정하는데 n_estimators 대신 부스팅 반복 횟수를 지정하는 `max_iter` 사용


In [11]:
#from sklearn.experimental import enable_hist_gradient_boosting  테스트 중이었는데 이제 임포트 안해도됨
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합 잘 억제하면서 그레이디언트 부스팅보다 조금 더 나은 성능

#### 특성 중요도

In [12]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)

result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1) # 랜덤으로 섞일 횟수 10

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [14]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1) #테스트 세트

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [15]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [16]:
hgb.score(test_input, test_target)  # 성능. 단일 결정 트리보다 좋은 결과

0.8723076923076923

In [ ]:
pip install xgboost

#### XGBoost

In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)  # 히스토그램 기반 그레이디언트 부스팅 사용
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM

In [8]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)  # 히스토그램 기반 그레이디언트 부스팅 사용
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
